In [10]:
import numpy as np
from scipy import signal
import math
import time
import copy
from random import randint
from scipy import optimize as opt
from scipy.stats import norm
from scipy.interpolate import interp1d
import pylab as pl

In [11]:
import pandas as pd
fd = pd.read_csv('E:/study/FIN-516/data_2.csv')

In [12]:
fd
# zero_cbp=fd['P(0,T)']
# p0t=np.zeros(61)
# for i in range(0,61,3):p0t[i]=zero_cbp[i/3]
# p0t
p0t = np.asarray(fd['P0t'])
p0t

array([1.        , 0.99357977, 0.98651179, 0.97931779, 0.97189891,
       0.9644466 , 0.95712165, 0.95001584, 0.94305276, 0.9361712 ,
       0.92950297, 0.92282553, 0.9161419 , 0.90961704, 0.90345529,
       0.8972356 , 0.89096215, 0.88491562, 0.87901314, 0.87305965,
       0.86724789, 0.86138259, 0.85565333, 0.84999466, 0.84434424,
       0.83868379, 0.8332117 , 0.82774254, 0.82227784, 0.8168129 ,
       0.81152803, 0.80624428, 0.8009632 , 0.79569612, 0.79059768,
       0.785502  , 0.7804105 , 0.77528472, 0.77038867, 0.76542963,
       0.76058538, 0.75569174, 0.75095883, 0.74622644, 0.74149593,
       0.73686498, 0.73238749, 0.7279117 , 0.72343883, 0.71905335,
       0.71481392, 0.71057673, 0.70634287, 0.70215895, 0.69806939,
       0.69402755, 0.68999007, 0.68595436, 0.68214074, 0.67828451,
       0.67438828, 0.6706499 , 0.66699691, 0.66330331, 0.65969395,
       0.6560872 , 0.65260034, 0.64911497, 0.6456319 , 0.64218939,
       0.63882396, 0.63549715, 0.63217221, 0.62888567, 0.62570

In [13]:
# for i in range(20):
#     p0t[(i*3):(i*3+4)]=np.linspace(p0t[(i*3)],p0t[(i*3+3)],4)

In [14]:
dt=1/4
#
r=np.zeros((81,81))
r[0,0]=-math.log(p0t[1])/dt
#
pi=np.zeros((81,81)) 
pi[0,0]=1
#
u=np.zeros((1,81))
u[0,0]=r[0,0]
u[0,0]

0.025763705212925005

In [15]:
sigma=np.zeros((1,81))
# fd = pd.read_csv('Desktop/capletsigma.csv')
# capletsigma=fd['IV']
# sigma[0,0:12]=capletsigma[2]
sigma = np.asarray(fd['Strip_vol'])
sigma

array([0.141     , 0.141     , 0.141     , 0.14099999, 0.141     ,
       0.16104726, 0.17197675, 0.18343948, 0.19484573, 0.20233944,
       0.21269898, 0.22301789, 0.23346243, 0.24249379, 0.25630775,
       0.26497697, 0.27370934, 0.25142561, 0.25798561, 0.2609581 ,
       0.26754581, 0.28112003, 0.28821592, 0.29476967, 0.30048518,
       0.23350115, 0.23246204, 0.23170285, 0.23096172, 0.27450265,
       0.27824542, 0.28084869, 0.28338079, 0.24107442, 0.24066051,
       0.2402461 , 0.23983789, 0.23488434, 0.2334195 , 0.23342114,
       0.23232129, 0.22745442, 0.22608485, 0.2252662 , 0.22441907,
       0.22340225, 0.2219765 , 0.2211648 , 0.22032385, 0.22642761,
       0.22545975, 0.22493508, 0.22440863, 0.22372793, 0.22288609,
       0.22220159, 0.22151633, 0.22132563, 0.21958098, 0.21944529,
       0.21925593, 0.2300533 , 0.22964509, 0.22979269, 0.22938744,
       0.22922733, 0.22887271, 0.22871529, 0.22855699, 0.22833085,
       0.22803456, 0.22780715, 0.22765121, 0.22742233, 0.22704

In [16]:
# for i in range(5,21,1):
#     sigma[0,((i-1)*3):(i*3)]=capletsigma[i]

In [17]:
def find_u(p0t,pi,u,sigma,i):
    estimate=0
    #print(pi)
    for j in range(len(pi)):
        #print(estimate)
        estimate=estimate+pi[j]*math.exp(-u*dt*math.exp((2*j-i)*math.sqrt(dt)*sigma))
    
    return (p0t-estimate)**2


In [18]:
for i in range(1,80):
    for j in range(i+1):
        if j==0: 
            pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)
        else:
            pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)+0.5*pi[i-1,j-1]*math.exp(-r[i-1,j-1]*dt)
    #print(i,j)
    a=opt.minimize(lambda u: find_u(p0t[i+1],pi[i,:],u,sigma[i],i),0.01, method='BFGS', tol=1e-16)
    u[0,i]=a.x
    for j in range(i+1):
        r[i,j]=u[0,i]*math.exp((2*j-i)*sigma[i]*math.sqrt(dt))
#u

In [19]:
#caplet_p=fd['Caplet price, maturity T+t']
#caplet_p
caplet_k=fd['Strike']
#caplet_k

In [20]:
def caplet(S,K,sigma,T,P,N,tau):
    d1=(np.log(S/K)+0.5*sigma**2*T)/(sigma*np.sqrt(T))
    d2=(np.log(S/K)-0.5*sigma**2*T)/(sigma*np.sqrt(T))
    V=N*tau*P*(S*norm.cdf(d1)-K*norm.cdf(d2))
    return (V)

def find_cap_vol(sigma,args):
    price=args[0]
    S=args[1]
    K=args[2]
    T=args[3]
    P=args[4]
    N=args[5]
    tau=args[6]
    cap_price=caplet(S,K,sigma,T,P,N,tau)
    return (cap_price-price)**2

In [21]:
# caplet_estimate=[]
# for i in range(2,80):
#     f=(pi[(i-1)*3,:].sum(axis=0)/pi[i*3,:].sum(axis=0)-1)*4
#     p=pi[i*3,:].sum(axis=0)
#     caplet_estimate.append(caplet(f,caplet_k[i],sigma[(i-1)*3],(i-1)*3*dt,p,1,0.25))

caplet_estimate=[]
caplet_estimate.append(0.00019874824)
caplet_estimate.append(0.00019874824)
for i in range(2,80):
    f=(pi[(i-1),:].sum(axis=0)/pi[i,:].sum(axis=0)-1)*4
    p=pi[i,:].sum(axis=0)
    caplet_estimate.append(caplet(f,caplet_k[i],sigma[(i-1)],(i-1)*dt,p,1,0.25))


caplet_estimate.__len__()

#caplet_p

80

In [22]:
caplet_p = caplet_estimate
new_sig=[]
new_sig.append(0.141)
new_sig.append(0.141)
new_sig.append(0.141)
for i in range(2,80):
    f=(pi[(i-1),:].sum(axis=0)/pi[i,:].sum(axis=0)-1)*4
    p=pi[i,:].sum(axis=0)
    #print(i)
    args=[caplet_p[i],f,caplet_k[i],i*dt,p,1,0.25]
    temp=opt.minimize(lambda sigma: find_cap_vol(sigma,args),0.4, method='BFGS', tol=1e-16)
    new_sig.append(temp.x[0])

In [23]:
#new_sig
# sigma[0,0:12]=new_sig[1]
# for i in range(5,21,1):
#     sigma[0,((i-1)*3):(i*3)]=capletsigma[i]
# print(sigma.__len__())
# print(new_sig.__len__())
# print(sigma)
# print(new_sig)

In [24]:
# for i in range(1,80):
#     for j in range(i+1):
#         if j==0: 
#             pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)
#         else:
#             pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)+0.5*pi[i-1,j-1]*math.exp(-r[i-1,j-1]*dt)
#     #a=opt.minimize(lambda u: find_u(p0t[i+1],pi[i,:],u,sigma[0,i],i),0.4, method='BFGS', tol=1e-16)
#     #u[0,i]=a.x
#     for j in range(i+1):
#         r[i,j]=u[0,i]*math.exp((2*j-i)*sigma[0,i]*math.sqrt(dt))
      

# i=80
# for j in range(i+1):
#     if j==0: 
#         pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)
#     else:
#         pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)+0.5*pi[i-1,j-1]*math.exp(-r[i-1,j-1]*dt)
# p=[]
# for i in range(61):
#     p.append(pi[i,:].sum(axis=0))

# error=p-p0t
# error

In [25]:
for i in range(1,80):
    for j in range(i+1):
        if j==0: 
            pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)
        else:
            pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)+0.5*pi[i-1,j-1]*math.exp(-r[i-1,j-1]*dt)
    a=opt.minimize(lambda u: find_u(p0t[i+1],pi[i,:],u,new_sig[i],i),0.01, method='BFGS', tol=1e-16)
    u[0,i]=a.x
    for j in range(i+1):
        r[i,j]=u[0,i]*math.exp((2*j-i)*new_sig[i]*math.sqrt(dt))
      

In [26]:
i=80
for j in range(i+1):
    if j==0: 
        pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)
    else:
        pi[i,j]=0.5*pi[i-1,j]*math.exp(-r[i-1,j]*dt)+0.5*pi[i-1,j-1]*math.exp(-r[i-1,j-1]*dt)
p=[]
for i in range(81):
    p.append(pi[i,:].sum(axis=0))

In [39]:
error=p-p0t
error
#np.savetxt("uu.csv", u, delimiter=",")
# np.savetxt("rtree1.csv", np.flip(r.transpose(),axis=0), delimiter=",")
# np.savetxt("pp1.csv", p, delimiter=",")
#np.savetxt("pi.csv", np.flip(pi.transpose(),axis=0), delimiter=",")
p = np.asarray(p)
#p[4:8].sum()

In [49]:
# # swaprate=0.029316032

# # vol=0.2569
# def swaption(S,K,sigma,T,tn):
#     d1=(np.log(S/K)+0.5*sigma**2*T)/(sigma*np.sqrt(T))
#     d2=(np.log(S/K)-0.5*sigma**2*T)/(sigma*np.sqrt(T))
#     fixed_leg = p[T:T+tn].sum()
#     V=(-S*norm.cdf(-d1)+K*norm.cdf(-d2))*fixed_leg
#     return (V)


# strike = [0.00630327202255,
#     0.0121836566202,
#     0.0176857705887,
#     0.022867656702,
#     0.0278724131908,
#     0.00604542339332,
#     0.011701959619,
#     0.0170292804789,
#     0.0221745003964,
#     0.0271316342751]   
# vols = [0.1737,0.2064,0.2145,0.2145,0.213,0.2108,0.2364,0.2365,0.2427,0.2328]
# tn = [4,8,12,16,20]
    
# v11 = swaption(strike[0],strike[0],vols[0],4,tn[0])
# v11

0.0033408069730958075

In [40]:
# v12 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v13 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v14 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v15 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v21 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v22 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v23 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v24 = swaption(strike[0],strike[0],vol[0],4,tn[0])
# v25
# fixed_leg=0.25*(pi[30,:].sum(axis=0)+pi[33,:].sum(axis=0)+pi[36,:].sum(axis=0)+
#                pi[42,:].sum(axis=0)+pi[45,:].sum(axis=0)+pi[48,:].sum(axis=0)+
#                pi[51,:].sum(axis=0)+pi[54,:].sum(axis=0)+pi[57,:].sum(axis=0)+pi[60,:].sum(axis=0))
# swaption_p=swaption(0.029316032,0.029316032,vol,2,fixed_leg,1,0.25)
# swaption_p

# value=np.zeros((61,61))
# coupon=1*swaprate/4
# value[60,:]=1+coupon
# for i in range(59,23,-1):
#     for j in range(i,-1,-1):
#         value[i,j]=(value[i+1,j+1]+value[i+1,j])/2/(1+r[i,j]*dt) 
#         if (i%3==0): 
#             value[i,j]=value[i,j]+coupon
# value[24,0:24]=value[24,0:24]-coupon

# i=24
# for j in range(24,-1,-1):
#     value[i,j]=max(1-value[i,j],0) 


# for i in range(23,-1,-1):
#     for j in range(i,-1,-1):
#         value[i,j]=(value[i+1,j+1]+value[i+1,j])/2/(1+r[i,j]*dt) 

# value[0,0]

# value[24,:]

# value[24,0:24]

NameError: name 'vol' is not defined